In [32]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [33]:
# original dataset with specific index
# data = pd.read_csv('data/adult.data', na_values=["?"])
# target_dataset = pd.DataFrame(data.iloc[1:10001,].values)
# shadow_dataset = pd.DataFrame(data.iloc[15001:30001,].values)
# attack_test_nonmembers = pd.DataFrame(data.iloc[11001:14001,].values)
# attack_test_members = pd.DataFrame(data.iloc[2001:5001,].values)

In [34]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [35]:
def old_transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column
    

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data
            print(x.iloc[:,j])

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    
    y=to_categorical(y)
    
    return x, y, dim

In [36]:
def transform_data(dataset, is_synthetic):
    

    le = LabelEncoder()
    dataset[10] = le.fit_transform(dataset[10].astype('str'))

    # normalize the values
    x_range = [i for i in range(10)]
    #dataset[x_range] = dataset[x_range]/dataset[x_range].max()


    x = dataset[x_range].values
    y = dataset[10].values
        
    
    dim = x.shape[1]
    
    x=np.array(x)
    y=np.array(y)

    y=to_categorical(y)
    
    return x, y, dim

In [37]:
def load_target_data(dataset, train_size, test_ratio, is_synthetic):
    x, y, dim = transform_data(dataset, is_synthetic)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:train_size,]
    testX = x[7000:,]
    trainY = y[0:train_size,]
    testY = y[7000:,]
    return (trainX, trainY), (testX, testY), dim

In [38]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [39]:
def build_simple_dnn(n_class,dim, channel=0):
    model = Sequential()
    
    model.add(Dense(600, input_dim=dim))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    #model.add(Dense(1024, kernel_regularizer=l2(0.00003)))
    #model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(512, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))

    model.add(Dense(256, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(128, kernel_regularizer=l2(0.00003)))
    model.add(Activation("tanh"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    #opt = SGD(lr=0.01, momentum=0.9)
    #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    act_layer=6
    
    return model, act_layer

In [40]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio, is_synthetic):
    x, y, _ = transform_data(dataset, is_synthetic)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio)
        print('shadow_i_trainX = ', len(trainX), 'shadow_i_trainY = ', len(trainY), 'shadow_i_testX = ', len(testX), 'shadow_i_testY = ', len(testY))
        
        np.savez(DATA_PATH + 'shadow_avila{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [41]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    
    train_accuracy=[]
    test_accuracy=[]

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_avila{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

        model, act_layer = build_simple_mlp(n_class,dim, channel)
            
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))
        train_accuracy.append((train_acc * 100.0))
        test_accuracy.append((test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))


    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    nonmembers = [item for sublist in nonmembers for item in sublist]
    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)
    shadow_accuracy = (train_accuracy, test_accuracy)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model, shadow_accuracy

In [42]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(learning_rate=0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [43]:
def attack_mlp(pix,d):

    model = Sequential()
    model.add(Dense(64, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.1))

#     model.add(Dense(32))
#     model.add(Activation("tanh"))
#     model.add(Dropout(0.01))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=1
    
    return model, act_layer

In [44]:
def prep_attack_train_data(n_attack_data):

    attack_mem = pd.DataFrame(n_attack_data[0][0])
    attack_nmem = pd.DataFrame(n_attack_data[0][1])
    
    attack_mem_status = pd.DataFrame(n_attack_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(n_attack_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(n_attack_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(n_attack_data[2][1])
    real_class_nmem.columns = ["y"]

    memdf = pd.concat([attack_mem,attack_nmem],axis=0)
    memdf = memdf.reset_index(drop=True)

    memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
    memstatus = memstatus.reset_index(drop=True)

    realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
    realclass = realclass.reset_index(drop=True)

    attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return attack_df

In [45]:
def prep_validation_data(attack_test_data):

    attack_mem = pd.DataFrame(attack_test_data[0][0])
    attack_nmem = pd.DataFrame(attack_test_data[0][1])
    
    attack_mem_status = pd.DataFrame(attack_test_data[1][0])
    attack_mem_status.columns = ["membership"]
    
    attack_nmem_status = pd.DataFrame(attack_test_data[1][1])
    attack_nmem_status.columns = ["membership"]
    
    real_class_mem = pd.DataFrame(attack_test_data[2][0])
    real_class_mem.columns = ["y"]
    
    real_class_nmem = pd.DataFrame(attack_test_data[2][1])
    real_class_nmem.columns = ["y"]
    
    mem_df = pd.concat([attack_mem,real_class_mem],axis=1)
    nmem_df = pd.concat([attack_nmem,real_class_nmem],axis=1)

#     memdf = pd.concat([attack_mem,attack_nmem],axis=0)
#     memdf = memdf.reset_index(drop=True)

#     memstatus =  pd.concat([attack_mem_status,attack_nmem_status],axis=0)
#     memstatus = memstatus.reset_index(drop=True)

#     realclass = pd.concat([real_class_mem,real_class_nmem],axis=0)
#     realclass = realclass.reset_index(drop=True)

#     attack_df = pd.concat([memdf,realclass,memstatus],axis=1)
    
    return mem_df, nmem_df

In [46]:
def load_attack_test_data(members, nonmembers, is_synthetic):
    memberX, memberY, _ = transform_data(members, is_synthetic)
    
    nonmemberX, nonmemberY, _ = transform_data(nonmembers, is_synthetic)
    
    return memberX, memberY, nonmemberX, nonmemberY

In [47]:
def prety_print_result(mem, pred):
    tn, fp, fn, tp = confusion_matrix(mem, pred).ravel()
    print('TP: %d     FP: %d     FN: %d     TN: %d' % (tp, fp, fn, tn))
    if tp == fp == 0:
        print('PPV: 0\nAdvantage: 0')
    else:
        print('PPV: %.4f\nAdvantage: %.4f' % (tp / (tp + fp), tp / (tp + fn) - fp / (tn + fp)))

    return tp, fp, fn, tn, (tp / (tp + fp)), (tp / (tp + fn) - fp / (tn + fp)), ((tp+tn)/(tp+tn+fp+fn)),  (tp / (tp + fn))

In [48]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])
        
        d=1
        pix = full_cx_data.shape[1]
        classifier, _ = attack_mlp(pix,d)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        pred_y = classifier.predict(checkmem_prediction_vals[c_indices])
        print(pred_y)
        c_pred_y = np.argmax(pred_y, axis=1)
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall

In [49]:
def shokri_attack(attack_df, mem_validation, nmem_validation):
    
    predicted_membership, predicted_nmembership, true_membership, TP_idx, TN_idx  = [], [], [], [], []

    class_val = np.unique(attack_df['y'])
    ncval=attack_df.shape[1]-1
    
    for c_val in class_val:

        print(c_val)
        
        filter_rec_all = attack_df[(attack_df['y'] == c_val)]
        filter_rec_idx = np.array(filter_rec_all.index)
        
        attack_feat = filter_rec_all.iloc[:, 0:ncval]
        attack_class = filter_rec_all['membership']
             
        d=1
        pix = attack_feat.shape[1]
        
        attack_model, _ = attack_mlp(pix,d)
        
       
        history = attack_model.fit(attack_feat, attack_class, epochs=EPS, batch_size=32, verbose=0)
        
        mcval=mem_validation.shape[1]-1
        
        
        check_mem_feat = mem_validation[mem_validation['y']==c_val]
        check_nmem_feat = nmem_validation[nmem_validation['y']==c_val]
        
        if (len(check_mem_feat)!=0) and (len(check_nmem_feat)!=0):
        
            check_mem_feat_idx =  np.array(check_mem_feat.index)


            check_nmem_feat_idx =  np.array(check_nmem_feat.index)

            #print(check_nmem_feat_idx)
            #print(np.argmax(mpred,axis=1)==0)


            mpred = attack_model.predict(np.array(check_mem_feat))    
            predicted_membership.append(np.argmax(mpred,axis=1) )

            nmpred = attack_model.predict(np.array(check_nmem_feat))    
            predicted_nmembership.append(np.argmax(nmpred,axis=1) )        



            TP_idx.append(check_mem_feat_idx[np.where(np.argmax(mpred,axis=1)==1)[0]])

            TN_idx.append(check_nmem_feat_idx[np.where(np.argmax(nmpred,axis=1)==0)[0]])

    pred_members = np.array([item for sublist in predicted_membership for item in sublist])
    pred_nonmembers = np.array([item for sublist in predicted_nmembership for item in sublist])
    
    TP_idx_list = np.array([item for sublist in TP_idx for item in sublist])
    TN_idx_list = np.array([item for sublist in TN_idx for item in sublist])
    
    members=np.array(list(pred_members))
    nonmembers=np.array(list(pred_nonmembers))
    
    pred_membership = np.concatenate([members,nonmembers])
    ori_membership = np.concatenate([np.ones(len(members)), np.zeros(len(nonmembers))])
    
    return pred_membership, ori_membership, TP_idx_list, TN_idx_list

In [50]:
def train_target_model(target_dataset, per_class_sample, epoch, act_layer, n_class, is_synthetic, train_size, channel=0, verbose=0, test_ratio=0.3):
    
    (target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, train_size, test_ratio, is_synthetic)
    target_model,_ = build_simple_mlp(n_class,dim, channel)
    #get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
    history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
    score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    _, train_acc = target_model.evaluate(target_trainX, target_trainY, verbose=VERBOSE)
    _, test_acc = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
    print('\n', "Target Train acc : ", (train_acc * 100.0),"Target Test acc : ", (test_acc * 100.0))
    #print('\n', 'Model test accuracy:', score[1])
    return target_model, dim

In [51]:
def prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic):
    members = []
    nonmembers = []

    memberX, memberY, nonmemberX, nonmemberY = load_attack_test_data(attack_test_members, attack_test_nonmembers, is_synthetic)

    # member
    target_model_member_pred = target_model.predict(memberX, batch_size=32)
    target_model_member_class = np.argmax(memberY, axis=1)
    target_model_member_pred = np.vstack(target_model_member_pred)
    #target_model_member_class = [item for sublist in target_model_member_class for item in sublist]
    members.append(np.ones(len(target_model_member_pred)))
    members = [item for sublist in members for item in sublist]


    # nonmember
    target_model_nonmember_pred = target_model.predict(nonmemberX, batch_size=32)
    target_model_nonmember_class = np.argmax(nonmemberY, axis=1)
    target_model_nonmember_pred = np.vstack(target_model_nonmember_pred)
    #target_model_nonmember_class = [item for sublist in target_model_nonmember_class for item in sublist]
    nonmembers.append(np.zeros(len(target_model_nonmember_pred)))
    nonmembers = [item for sublist in nonmembers for item in sublist]

    full_attack_test_pred_val = (target_model_member_pred, target_model_nonmember_pred)
    full_attack_test_mem_status = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    full_attack_test_class_status = (np.array(target_model_member_class),np.array(target_model_nonmember_class))

    print('\n pred', full_attack_test_pred_val)
    print('\n class', full_attack_test_class_status)
    print('\n mem status', full_attack_test_mem_status)

    attack_test_data = (full_attack_test_pred_val, full_attack_test_mem_status,full_attack_test_class_status)
    
    return attack_test_data

In [246]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [25]:
# original dataset with random index
train_size = 7000
attack_test_size = 250
data = pd.read_csv('data/avila-tr.txt', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = pd.read_csv('data/avila-ts.txt', na_values=["?"], header=None)
shadow_dataset = df_rest.sample(n = 7000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [26]:
# trian target model
per_class_sample=150
channel=0   
EPS=200
act_layer=3
n_class = 12
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic, train_size)


 Target Train acc :  97.10000157356262 Target Test acc :  92.86666512489319


In [204]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 750

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(n_shadow_models, n_class, dim, channel)

shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_t

In [205]:
#train attack model
attack_test_data = prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

8/8 [==============================] - 0s 2ms/step

 pred (array([[9.8714331e-13, 6.9560365e-29, 0.0000000e+00, ..., 2.6858010e-24,
        7.9548124e-28, 9.0669403e-16],
       [9.9999827e-01, 1.3162569e-11, 9.4170819e-15, ..., 3.0618432e-07,
        1.3237573e-07, 4.1405130e-07],
       [8.4214187e-01, 1.1881189e-06, 4.3991413e-06, ..., 6.5386434e-06,
        1.2312449e-05, 8.1916196e-06],
       ...,
       [2.0310460e-01, 5.5409937e-06, 3.1311571e-04, ..., 5.8683008e-04,
        7.6346754e-05, 6.0785293e-05],
       [4.1725137e-08, 1.8437559e-13, 3.0010899e-10, ..., 6.9823525e-09,
        4.5893788e-03, 9.9534422e-01],
       [3.5555948e-02, 7.4192535e-08, 6.0806116e-09, ..., 3.6230577e-07,
        1.5032805e-06, 1.0204015e-05]], dtype=float32), array([[9.9714029e-01, 1.8152552e-09, 1.3582187e-12, ..., 2.2745754e-08,
        2.8992619e-11, 2.2385427e-08],
       [9.7578484e-01, 5.7244280e-09, 2.1229867e-02, ..., 5.0547860e-06,
        1.4580105e-03, 1.5208459e-03],
       [1.331294

In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [247]:
# synthetic dataset
train_size = 500
attack_test_size = 250
target_dataset = pd.read_csv('data/avila_sds.csv', na_values=["?"], header=None)
df = pd.read_csv('data/avila-ts.txt', na_values=["?"], header=None)
shadow_dataset = df.sample(n = 5000, replace = False)
df_rest = df.loc[~df.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members = pd.read_csv('data/avila-tr.txt', na_values=["?"], header=None)
attack_test_members = attack_test_members.sample(n=attack_test_size, replace=False)

In [29]:
train_size = 7000
target_dataset = pd.read_csv('data/avila_sds_sdv_copulagan.csv', na_values=["?"], header=None)

In [30]:
# trian target model
per_class_sample=150
channel=0   
EPS=200
act_layer=3
n_class = 12
is_synthetic = True
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, per_class_sample, EPS, act_layer, n_class, is_synthetic, train_size)


 Target Train acc :  57.3285698890686 Target Test acc :  41.76666736602783


In [249]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 750

load_shadow_data(shadow_dataset, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(n_shadow_models, n_class, dim, channel)

shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_trainX =  525 shadow_i_trainY =  525 shadow_i_testX =  225 shadow_i_testY =  225
shadow_i_t

In [250]:
#train attack model
attack_test_data = prepare_attack_test_data(attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list = shokri_attack(attack_train_df, mem_validation, nmem_validation)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

8/8 [==============================] - 0s 4ms/step

 pred (array([[8.29200362e-05, 6.31923536e-09, 8.15046860e-06, ...,
        1.09960241e-09, 2.21771259e-07, 3.28547202e-07],
       [1.63710961e-10, 1.48050272e-19, 1.82836905e-11, ...,
        4.57471699e-10, 9.97439981e-01, 1.82544964e-03],
       [1.00084848e-03, 1.78367738e-08, 3.51257114e-07, ...,
        1.08447065e-08, 1.36180915e-08, 3.47947719e-07],
       ...,
       [9.67245042e-01, 4.44265025e-09, 1.31724098e-09, ...,
        5.87845648e-08, 1.52431720e-10, 1.77634161e-06],
       [9.61077929e-01, 1.22125229e-07, 1.76585491e-09, ...,
        3.11442790e-07, 7.19873050e-09, 2.58238629e-06],
       [4.50925291e-01, 4.31745957e-06, 1.80087522e-06, ...,
        1.57275008e-05, 1.20057200e-06, 4.96224791e-04]], dtype=float32), array([[4.3696832e-02, 1.8540393e-07, 2.5460375e-08, ..., 6.5816877e-08,
        6.3296968e-10, 7.4345695e-07],
       [5.0678100e-03, 1.7937796e-07, 8.4155627e-06, ..., 9.6001156e-07,
        2.2927592e-

In [ ]:
# --------------------------------------------Synthetic Data-------------------------------------------------------------#

In [ ]:
# --------------------------------------------Overfitting Experiment-----------------------------------------------------#

In [166]:
# original dataset with random index
data = pd.read_csv('data/avila-tr.txt', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 10000, replace = False)
df_rest = pd.read_csv('data/avila-ts.txt', na_values=["?"], header=None)
shadow_dataset = df_rest.sample(n = 7000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = 1500, replace = False)
attack_test_members =  target_dataset.iloc[:3000,:].sample(n = 1500, replace = False)

In [167]:
def load_target_data(dataset, training_data_size, test_ratio, is_synthetic):
    x, y, dim = transform_data(dataset, is_synthetic)
    
    #trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    trainX = x[0:training_data_size,]
    testX = x[5000:,]
    trainY = y[0:training_data_size,]
    testY = y[5000:,]
    return (trainX, trainY), (testX, testY), dim

In [169]:
# trian target model
training_data_size = [2,5,10,15,20,25,50,100,200,500,1000,2000,5000]
per_class_sample=150
channel=0   
EPS=200
act_layer=3
n_class = 12
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

accuracy_df = pd.DataFrame()
accuracy_df['training data size'] = np.nan
accuracy_df['training accuracy'] = np.nan
accuracy_df['test accuracy'] = np.nan
accuracy_df['error'] = np.nan

data = pd.read_csv('data/avila-tr.txt', na_values=["?"], header=None)
data.dropna(inplace=True)

for j in training_data_size:
    print('\n Training Data Size: ', j, '\n')
    for i in range(10):
        target_dataset = data.sample(n = 10000, replace = False)
        (target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, j, test_ratio, is_synthetic)
        target_model,_ = build_simple_mlp(n_class,dim, channel)
        #get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
        history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
        score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
        _, train_acc = target_model.evaluate(target_trainX, target_trainY, verbose=VERBOSE)
        _, test_acc = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
        print('Iteration ', i, "Target Train acc : ", (train_acc * 100.0),"Target Test acc : ", (test_acc * 100.0))
        accuracy_df = accuracy_df.append({'training data size':j, 'training accuracy' : (train_acc * 100.0), 'test accuracy': (test_acc * 100.0), 'error': ((train_acc * 100.0)-(test_acc * 100.0))}, ignore_index=True)
        #print('\n', 'Model test accuracy:', score[1])


 Training Data Size:  2 

Iteration  0 Target Train acc :  100.0 Target Test acc :  16.760000586509705
Iteration  1 Target Train acc :  100.0 Target Test acc :  39.05999958515167
Iteration  2 Target Train acc :  100.0 Target Test acc :  17.560000717639923
Iteration  3 Target Train acc :  100.0 Target Test acc :  7.5599998235702515
Iteration  4 Target Train acc :  100.0 Target Test acc :  8.720000088214874
Iteration  5 Target Train acc :  100.0 Target Test acc :  32.67999887466431
Iteration  6 Target Train acc :  100.0 Target Test acc :  33.25999975204468
Iteration  7 Target Train acc :  100.0 Target Test acc :  27.52000093460083
Iteration  8 Target Train acc :  100.0 Target Test acc :  39.899998903274536
Iteration  9 Target Train acc :  100.0 Target Test acc :  4.899999871850014

 Training Data Size:  5 

Iteration  0 Target Train acc :  100.0 Target Test acc :  22.87999987602234
Iteration  1 Target Train acc :  100.0 Target Test acc :  34.299999475479126
Iteration  2 Target Train acc

Iteration  2 Target Train acc :  97.39999771118164 Target Test acc :  70.21999955177307
Iteration  3 Target Train acc :  96.8999981880188 Target Test acc :  71.17999792098999
Iteration  4 Target Train acc :  96.20000123977661 Target Test acc :  69.08000111579895
Iteration  5 Target Train acc :  98.1000006198883 Target Test acc :  70.4200029373169
Iteration  6 Target Train acc :  96.29999995231628 Target Test acc :  69.88000273704529
Iteration  7 Target Train acc :  97.60000109672546 Target Test acc :  72.03999757766724
Iteration  8 Target Train acc :  96.39999866485596 Target Test acc :  71.43999934196472
Iteration  9 Target Train acc :  96.60000205039978 Target Test acc :  71.29999995231628

 Training Data Size:  2000 

Iteration  0 Target Train acc :  94.30000185966492 Target Test acc :  74.72000122070312
Iteration  1 Target Train acc :  96.5499997138977 Target Test acc :  79.37999963760376
Iteration  2 Target Train acc :  98.25000166893005 Target Test acc :  78.75999808311462
Iterat

In [172]:
#accuracy_df =pd.read_csv('data/adult_overfitting_test_trainsize')
training_data_size = [2,5,10,15,20,25,50,100,200,500,1000,2000,5000]

avg_error_ci95_df = pd.DataFrame()
avg_error_ci95_df['training data size'] = np.nan
avg_error_ci95_df['average training accuracy'] = np.nan
avg_error_ci95_df['average test accuracy'] = np.nan
avg_error_ci95_df['average error'] = np.nan
avg_error_ci95_df['ci95 low'] = np.nan
avg_error_ci95_df['ci95 high'] = np.nan

for i in training_data_size:
    error = np.array(accuracy_df.loc[accuracy_df['training data size'] == i, 'error'])
    training_accuracy = np.array(accuracy_df.loc[accuracy_df['training data size'] == i, 'training accuracy'])
    test_accuracy = np.array(accuracy_df.loc[accuracy_df['training data size'] == i, 'test accuracy'])
    ci95 = stats.t.interval(alpha=0.95, df=len(error)-1, loc=np.mean(error), scale=stats.sem(error))
    row = pd.DataFrame({'training data size': [i], 'average training accuracy': \
            [np.mean(training_accuracy)], 'average test accuracy': [np.mean(test_accuracy)], 'average error': [np.mean(error)],\
                                                  'ci95 low': [ci95[0]], 'ci95 high': [ci95[1]]})
    avg_error_ci95_df = pd.concat([avg_error_ci95_df, row], axis=0, ignore_index=True)

In [174]:
avg_error_ci95_df

,training data size,average training accuracy,average test accuracy,average error,ci95 low,ci95 high
0,2.0,100.000000,22.792000,77.208000,67.667063,86.748937
1,5.0,100.000000,24.004000,75.996000,69.007891,82.984108
2,10.0,100.000000,33.382000,66.618000,62.461118,70.774882
3,15.0,100.000000,35.232000,64.768000,62.960384,66.575616
4,20.0,100.000000,37.382000,62.618000,58.943850,66.292150
5,25.0,100.000000,39.334000,60.666000,57.831477,63.500523
6,50.0,100.000000,45.028000,54.972000,52.339900,57.604101
7,100.0,98.900000,50.298000,48.602000,45.522708,51.681293
8,200.0,99.800000,56.770001,43.029999,41.886576,44.173422
9,500.0,98.499999,64.207999,34.292000,33.395389,35.188611


In [ ]:
# --------------------------------------------Overfitting Experiment-----------------------------------------------------#